In [104]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import tensorflow as tf
import numpy as np
import os,glob,cv2
from os.path import dirname
import sys,argparse
from sklearn.utils import shuffle

In [71]:
image_size=128
num_channels=3
label =[]

In [198]:
files = []
images = []
labels = []
testdata_path="./data/potholes/Test-Data/"
for (dirpath, dirnames, filenames) in os.walk(testdata_path):
    print(dirnames)
    for file in filenames:
        imagefile = os.path.join(dirpath, file)
        if (os.path.isdir(imagefile)):
            continue
        image = cv2.imread(imagefile)
        image = cv2.resize(image, (image_size, image_size),0,0, cv2.INTER_LINEAR)
        image = image.astype(np.float32)
        image = np.multiply(image, 1.0 / 255.0)
        images.append(image)
        folder = os.path.basename(dirpath)
        if (folder == 'Negative'):
            label = [1,0]
        if (folder == 'Positive'):
            label =[0,1]
        labels.append(np.asarray(label))
images = np.array(images)
labels = np.array(labels)



['Positive', 'Negative']
[]
[]


In [199]:
images, labels = shuffle(images, labels)

In [200]:
x_batch = images.reshape(len(labels), image_size,image_size,num_channels)
x_labels = labels.reshape(len(labels), 2)

In [201]:
## Let us restore the saved model 
sess = tf.Session()
# Step-1: Recreate the network graph. At this step only graph is created.
saver = tf.train.import_meta_graph('./SavedModel/multiclass_potholes_model.meta')
# Step-2: Now let's load the weights saved using the restore method.
saver.restore(sess, tf.train.latest_checkpoint('./SavedModel/'))

INFO:tensorflow:Restoring parameters from ./SavedModel/multiclass_potholes_model


In [202]:
# Accessing the default graph which we have restored
graph = tf.get_default_graph()

# Now, let's get hold of the op that we can be processed to get the output.
# In the original network y_pred is the tensor that is the prediction of the network
y_pred = graph.get_tensor_by_name("y_pred:0")

## Let's feed the images to the input placeholders
x= graph.get_tensor_by_name("x:0") 
y_true = graph.get_tensor_by_name("y_true:0") 
y_test_images = np.zeros((1, 2)) 

In [203]:
### Creating the feed_dict that is required to be fed to calculate y_pred 
feed_dict_testing = {x: x_batch, y_true: y_test_images}
result=sess.run(y_pred, feed_dict=feed_dict_testing)
# result is of this format [probabiliy_of_rose probability_of_sunflower]
print(result)

[[0.72672504 0.273275  ]
 [0.8884291  0.11157086]
 [0.7885622  0.21143782]
 [0.6801947  0.31980535]
 [0.73788875 0.26211125]
 [0.74007094 0.259929  ]
 [0.03844582 0.96155417]
 [0.031732   0.968268  ]
 [0.87903565 0.12096436]
 [0.77495337 0.22504662]
 [0.8266009  0.1733991 ]
 [0.85134405 0.148656  ]
 [0.57217085 0.42782918]
 [0.7791743  0.22082572]
 [0.02425201 0.975748  ]
 [0.8294899  0.17051007]
 [0.05045214 0.9495479 ]
 [0.03238932 0.9676107 ]
 [0.44099414 0.5590059 ]
 [0.79599714 0.20400283]
 [0.87539655 0.1246035 ]
 [0.01387409 0.9861259 ]
 [0.10763646 0.89236355]
 [0.84394395 0.15605603]
 [0.05858733 0.9414126 ]
 [0.02068738 0.97931266]
 [0.84623367 0.15376635]
 [0.4288847  0.57111526]
 [0.8039143  0.19608572]
 [0.01923133 0.98076874]
 [0.02397296 0.9760271 ]
 [0.02100084 0.97899914]
 [0.0472468  0.9527532 ]
 [0.02162216 0.9783779 ]
 [0.02989602 0.9701039 ]
 [0.01629788 0.98370206]
 [0.7898531  0.21014695]
 [0.7153577  0.2846423 ]
 [0.8284856  0.1715144 ]
 [0.78210807 0.21789193]


In [204]:
print(np.c_[result, labels])

[[0.72672504 0.27327499 1.         0.        ]
 [0.88842911 0.11157086 1.         0.        ]
 [0.78856218 0.21143782 1.         0.        ]
 [0.68019468 0.31980535 1.         0.        ]
 [0.73788875 0.26211125 1.         0.        ]
 [0.74007094 0.259929   1.         0.        ]
 [0.03844582 0.96155417 0.         1.        ]
 [0.031732   0.96826798 0.         1.        ]
 [0.87903565 0.12096436 1.         0.        ]
 [0.77495337 0.22504662 1.         0.        ]
 [0.82660091 0.17339911 1.         0.        ]
 [0.85134405 0.148656   1.         0.        ]
 [0.57217085 0.42782918 1.         0.        ]
 [0.77917433 0.22082572 1.         0.        ]
 [0.02425201 0.975748   0.         1.        ]
 [0.82948989 0.17051007 1.         0.        ]
 [0.05045214 0.94954789 0.         1.        ]
 [0.03238932 0.96761072 0.         1.        ]
 [0.44099414 0.55900592 1.         0.        ]
 [0.79599714 0.20400283 1.         0.        ]
 [0.87539655 0.1246035  1.         0.        ]
 [0.01387409 

In [212]:
auc, update_op = tf.metrics.auc(
    labels,
    result,
    weights=None,
    num_thresholds=10,
    metrics_collections=None,
    updates_collections=None,
    curve='ROC',
    name=None,
    summation_method='trapezoidal'
)

with tf.Session() as sess:
    sess.run(tf.local_variables_initializer())
    print("tf auc: {}".format(sess.run([auc, update_op])))

tf auc: [0.0, 0.98525846]


In [213]:
import numpy as np
from sklearn.metrics import roc_auc_score
y_true = np.array([0, 0, 1, 1])
y_scores = np.array([0.1, 0.4, 0.35, 0.8])
print("sklearn auc: {}".format(roc_auc_score(labels, result)))

sklearn auc: 0.9970443349753695
